In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

In [2]:
station_day = pd.read_csv('station_day.csv\station_day.csv')
stations = pd.read_csv('station_day.csv\stations.csv')

print("Station day shape:", station_day.shape)
print("Stations shape:", stations.shape)

Station day shape: (108035, 16)
Stations shape: (230, 5)


In [3]:
print("Common columns:", set(station_day.columns).intersection(stations.columns))

if 'station_id' in station_day.columns and 'station_id' in stations.columns:
    data = station_day.merge(stations, on='station_id', how='left')
else:
    data = station_day.copy()

print("Merged shape:", data.shape)

Common columns: {'StationId'}
Merged shape: (108035, 16)


In [4]:
drop_cols = ['StationId', 'StationName', 'state', 'city', 'location', 'Date', 'date']
drop_cols = [c for c in drop_cols if c in data.columns]

data = data.drop(columns=drop_cols, errors='ignore')

In [5]:
data.shape

(108035, 14)

### Handling Missing Values

In [6]:
missing_percent = data.isna().mean().sort_values(ascending=False)
print("\nMissing value percentages:\n", missing_percent)

cols_high_missing = missing_percent[missing_percent > 0.7].index.tolist()
print("\nColumns with >70% missing:", cols_high_missing)


Missing value percentages:
 Xylene        0.788050
NH3           0.445272
PM10          0.395298
Toluene       0.358236
Benzene       0.291156
O3            0.236664
SO2           0.233295
PM2.5         0.200167
AQI_Bucket    0.194474
AQI           0.194474
NO            0.158338
NO2           0.153163
NOx           0.143472
CO            0.120313
dtype: float64

Columns with >70% missing: ['Xylene']


In [7]:
# Create two datasets:
#   1. one including xylene (after imputation)
#   2. one excluding xylene

data_with_xylene = data.copy()
data_without_xylene = data.drop(columns=cols_high_missing, errors='ignore')

In [8]:
# Drop AQI bucket to avoid data leakage

if 'AQI_Bucket' in data_with_xylene.columns:
    data_with_xylene.drop(columns=['AQI_Bucket'], inplace=True)
if 'AQI_Bucket' in data_without_xylene.columns:
    data_without_xylene.drop(columns=['AQI_Bucket'], inplace=True)

In [9]:
# numeric columns

numeric_cols_with = data_with_xylene.select_dtypes(include=[np.number]).columns
numeric_cols_without = data_without_xylene.select_dtypes(include=[np.number]).columns

In [10]:
# SimpleImputer for median filling
imputer = SimpleImputer(strategy='median')

data_with_xylene[numeric_cols_with] = imputer.fit_transform(data_with_xylene[numeric_cols_with])
data_without_xylene[numeric_cols_without] = imputer.fit_transform(data_without_xylene[numeric_cols_without])

In [11]:
# Handle categorical columns if any i.e., one-hot encoding
cat_cols_with = data_with_xylene.select_dtypes(exclude=[np.number]).columns
cat_cols_without = data_without_xylene.select_dtypes(exclude=[np.number]).columns

if len(cat_cols_with) > 0:
    data_with_xylene = pd.get_dummies(data_with_xylene, columns=cat_cols_with, drop_first=True)
if len(cat_cols_without) > 0:
    data_without_xylene = pd.get_dummies(data_without_xylene, columns=cat_cols_without, drop_first=True)

### Scaling

In [12]:
if 'AQI' not in data_with_xylene.columns:
    raise ValueError("❌ 'AQI' column not found!")

X_with = data_with_xylene.drop(columns=['AQI'])
y_with = data_with_xylene['AQI']

X_without = data_without_xylene.drop(columns=['AQI'])
y_without = data_without_xylene['AQI']

In [13]:
# Standardize features
scaler_with = StandardScaler()
scaler_without = StandardScaler()

X_with_scaled = pd.DataFrame(
    scaler_with.fit_transform(X_with),
    columns=X_with.columns
)

X_without_scaled = pd.DataFrame(
    scaler_without.fit_transform(X_without),
    columns=X_without.columns
)

In [14]:
import pickle

# Save the scalers
pickle.dump(scaler_with, open("scaler_with.pkl", "wb"))
pickle.dump(scaler_without, open("scaler_without.pkl", "wb"))
print("✅ Scalers saved: scaler_with.pkl, scaler_without.pkl")

✅ Scalers saved: scaler_with.pkl, scaler_without.pkl


In [60]:
X_with_scaled['AQI'] = y_with.values
X_without_scaled['AQI'] = y_without.values

X_with_scaled.to_csv("cleaned_with_xylene.csv", index=False)
X_without_scaled.to_csv("cleaned_without_xylene.csv", index=False)

In [61]:
print("\n✅ Feature Engineering Completed Successfully!")
print("With Xylene:", X_with_scaled.shape)
print("Without Xylene:", X_without_scaled.shape)
print("Files saved as: cleaned_with_xylene.csv & cleaned_without_xylene.csv")


✅ Feature Engineering Completed Successfully!
With Xylene: (108035, 13)
Without Xylene: (108035, 12)
Files saved as: cleaned_with_xylene.csv & cleaned_without_xylene.csv


In [15]:
df = pd.read_csv("cleaned_without_xylene.csv")
df.describe()

,PM2.5,PM10,NO,NO2,NOx,NH3,CO,SO2,O3,Benzene,Toluene,AQI
count,1.080350e+05,1.080350e+05,1.080350e+05,1.080350e+05,1.080350e+05,1.080350e+05,1.080350e+05,1.080350e+05,1.080350e+05,1.080350e+05,1.080350e+05,108035.000000
mean,7.050510e-17,2.020445e-16,-1.841551e-17,-1.210162e-17,2.315093e-17,1.515333e-16,9.418218e-17,1.667919e-16,2.809681e-16,1.578472e-17,1.042449e-16,170.463294
std,1.000005e+00,1.000005e+00,1.000005e+00,1.000005e+00,1.000005e+00,1.000005e+00,1.000005e+00,1.000005e+00,1.000005e+00,1.000005e+00,1.000005e+00,119.370472
min,-1.090495e+00,-1.473675e+00,-6.590500e-01,-1.244974e+00,-9.291615e-01,-1.412085e+00,-3.708206e-01,-1.001804e+00,-1.060378e+00,-2.893726e-01,-4.730584e-01,8.000000
25%,-5.493268e-01,-4.219414e-01,-4.789801e-01,-6.192053e-01,-5.504615e-01,-2.768780e-01,-2.270768e-01,-4.649354e-01,-4.028565e-01,-2.427785e-01,-3.846657e-01,96.000000
50%,-2.814190e-01,-2.223776e-01,-3.376752e-01,-2.490166e-01,-2.957775e-01,-1.524059e-01,-1.491140e-01,-2.213803e-01,-1.622218e-01,-1.612388e-01,-2.933680e-01,132.000000
75%,1.213830e-01,1.198446e-02,-1.098585e-02,3.013233e-01,1.408915e-01,-2.366011e-02,-4.191516e-02,9.463030e-02,1.343473e-01,-4.051766e-02,-1.464617e-01,211.000000
max,1.337508e+01,8.776037e+00,1.403384e+01,1.516050e+01,1.018072e+01,2.096564e+01,4.246242e+01,1.607674e+01,2.699395e+01,4.789636e+01,1.840274e+01,2049.000000
